# Performing a molecular dynamics simulation with OpenMM

In this exercise, we will perform a molecular dynamics simulations of a protein-ligand complex prepared in a previous exercise. It is adapted from [talktorial 19 of TeachOpenCADD](https://github.com/volkamerlab/teachopencadd/tree/master/teachopencadd/talktorials/T019_md_simulation), a platform that aims to teach domain-specific skills and to provide pipeline templates as starting points for research projects.

When you are done with this exercise, save it under your shared exercises folder on Google Drive. It will be graded as satisfactory or unsatisfactory based on correctly completing the sections after `-->`. Do not remove the symbol `-->`.

# Part 0 - Setting up the required software

The following code cells will install all required packages, if you are working on [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb). Installing the [condacolab](https://github.com/jaimergp/condacolab) package will restart the kernel, which is intended. This notebook can also be used on a local computer but requires considerable computing power.

In [ ]:
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

In [ ]:
!git clone https://github.com/CCBatIIT/MotorRow.git

try:
    import condacolab
    from google.colab import files
    from IPython.display import clear_output
    condacolab.check()
    !conda install -q -y -c conda-forge openmm cudatoolkit py3Dmol mdtraj
except ModuleNotFoundError:
    on_colab = False
else:
    #check if installation was succesful
    try:
        import openmm
        on_colab = True
        clear_output()  # clear the excessive installation outputs
        print("Dependencies successfully installed!")
    except ModuleNotFoundError:
        print("Error while installing dependencies!")

### Mount Google drive

In [ ]:
import os

# Check for directory and input files
if not os.path.isdir('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

# Part 1 - Molecular dynamics simulation

## Molecular dynamics

Molecular dynamics is a computational method analyzing the movements and interactions of atoms and molecules of a defined system. The method stems from theoretical physics, where it was developed in the 1950s (Alder and Wainwright in [_J Chem Phys_ (1959), **31**(2), 459](https://doi.org/10.1063/1.1730376)), although the ideas behind it can be dated much earlier:

> An intelligence which could, at  any moment, comprehend all the forces by  which nature is animated and the  respective positions of the  beings of which it is  composed, and moreover, if this intelligence were far-reaching enough to subject these data to  analysis, it would encompass in that formula both the movements of the  largest bodies in  the universe and those of the lightest atom: to it nothing would be uncertain, and the  future, as well as the past, would be present to its eyes. The human mind offers us, in the perfection which it has  given to  astronomy, a faint sketch of this intelligence. (Pierre-Simon Laplace, 1820)


Let us just take this statement by Laplace as the ideological substrate underneath molecular dynamics simulations. In other terms, we can approximate the behavior of a physical system by knowing the characteristics of its components and applying Newton's laws of motion. By solving the equations of motion, we can obtain a molecular trajectory of the system, which is a series of snapshots with the positions and velocities of all its particles, as well as its potential energy. To do so, we define functions, called force fields, which provide an approximate description of all the forces applied to each particle in the system. We then use numerical integrators to solve the initial value problem for the system and obtain the trajectory. As it sounds, the process requires quite a bit of processing power and it was only few years ago that MD started seeing a more widespread use, especially in the field of computational chemistry and biology, as well as in drug discovery ([_J Med Chem_ (2016), **59**(9), 4035‐4061](https://doi.org/10.1021/acs.jmedchem.5b01684)).

![MD_rotor_250K_1ns.gif](https://github.com/volkamerlab/teachopencadd/raw/d1ded86bb2c82ef088cc5145d0bcb997f6eab7dd/teachopencadd/talktorials/018_md_simulation/images/MD_rotor_250K_1ns.gif)

**Figure 1**: Molecular dynamics simulation of the rotation of a supramolecule composed of three molecules in a confined nanoscopic pore (Palma et al. via [Wikimedia](https://commons.wikimedia.org/w/index.php?curid=34866205)).

### MD simulations and drug design

MD simulations give valuable insights into the highly dynamic process of ligand binding to their target. When a ligand (or a drug) approaches a macromolecule (protein) in solution, it encounters a structure in constant motion. Also, ligands may induce conformational changes in the macromolecule that can best accommodate the small molecule. Such conformations may not be discovered with static methods. Accordingly, binding sites that are not observed in static ligand-free structures, but can be discovered with MD simulations, are sometimes called *cryptic binding sites* ([_J Med Chem_ (2016), **59**(9), 4035‐4061](https://doi.org/10.1021/acs.jmedchem.5b01684)). The identification of such binding sites with MD simulation can kickstart new drug discovery campaigns. Later in the drug discovery process, MD simulations can also be used to estimate the quality of computationally identified small molecules before performing more costly and time-intensive *in vitro* tests. Altogether, MD simulations pose a valuable asset in computational drug design.

We will now proceed to perform an MD simulation using the molecular dynamics engine [OpenMM](https://github.com/openmm/openmm), a high performance toolkit for molecular simulation. It is open source and can be used as application or library. We will use it as Python library.

## Loading the system

In a previous lab, we created an [OpenMM System](http://docs.openmm.org/development/api-python/generated/openmm.openmm.System.html#openmm.openmm.System) and set up the simulation. Here we will load it and perform a molecular dynamics simulation.

If you did not complete exercise 6, you will need to [download example results](https://github.com/daveminh/Chem456-2024F/blob/main/exercises/ADORA2A-LUF5448.tar.gz) and decompress the files into your shared Google drive folder.

--> Modify the paths below to point to your [OpenMM System](http://docs.openmm.org/development/api-python/generated/openmm.openmm.System.html#openmm.openmm.System) and output directory.

In [ ]:
import os

base_dir = os.path.join('/content','drive','MyDrive','Classes','Chem 456 2024F','students','dminh','exercises')
system_prefix = 'ADORA2A-LUF5448'
state_fn = os.path.join(base_dir, '06-System_Preparation', system_prefix + '.xml')
pdb_fn = os.path.join(base_dir, '06-System_Preparation', system_prefix + '.pdb')
openmm_dir = os.path.join(base_dir, '08-OpenMM-' + system_prefix)

if not os.path.isfile(state_fn):
  raise ValueError(f'Input file {state_fn} not found!')
if not os.path.isfile(pdb_fn):
  raise ValueError(f'Input file {pdb_fn} not found!')
if not os.path.isdir(openmm_dir):
  os.mkdir(openmm_dir)

## Setting up the simulation

Now we have loaded the system, we can set up a simulation. MotorRow is a wrapper around OpenMM that equilibrates a membrane system. The next code block creates a MotorRow instance.

In [ ]:
%cd MotorRow
from MotorRow import MotorRow
MR = MotorRow(state_fn, pdb_fn, openmm_dir)

MotorRow can be run with a single command, but I'll break down the steps so that you can think about them.

## Energy minimization
While everything is set up, we need to minimize the energy of the system to get a low energy starting configuration. This is important to decrease the chance of simulation failures due to severe atom clashes. The energy minimized system is saved.

In [ ]:
state_fn_o, pdb_fn_o = state_fn, pdb_fn
state_fn, pdb_fn = MR._minimize(pdb_fn)

#### --> How much does minimization decrease the energy of the system?

In [ ]:
# This visualization shows how minimization changes the system
ligand_name = "UNK"

import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(pdb_fn_o, "r").read(),'pdb')
view.addModel(open(pdb_fn, "r").read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.setStyle({'resn':ligand_name}, {'stick': {}})
# Show water and lipid near the protein
view.addStyle({'model':0, 'resn':'HOH', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'sphere': {'radius':'0.2', 'color':'red' }})
view.addStyle({'model':1, 'resn':'HOH', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'sphere': {'radius':'0.2', 'color':'blue'}})
view.addStyle({'model':0, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'stick': {'color':'red'}})
view.addStyle({'model':1, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'stick': {'color':'blue'}})
view.zoomTo()
view.show()

## Equilibration

Once the minimization has finished, we can start equilibration. Equilibration is needed because the initial minimized configuration of the system (suitable for T=0 K) is not representative of a typical configuration in the thermodynamic state. Throwing out an equilibration period reduces bias in estimates of expectation values.

that are  In this lab, we will perform an equilibration process suitable for membrane proteins. EquiliThis can be considered an "equilibration" of the system that is thrown out to minimize bias in estimated expectation values. The results are saved in an .dcd file, which contains the coordinates of all the atoms at a given time point. Together with the PDB file of the energy minimized system written before, it gives us all the information needed for later analysis.

**Note**: This lab will only generate a 20 fs trajectory, if not on Google Colab. However, if you have a good GPU available, you can also increase the simulation time.

### Step 1: NVT with restraints

This step is in the NVT ensemble; there are a constant number of particles, volume, and temperature. There are heavy restraints on the protein and on the Z coordinates of the membrane, allowing the water to equilibrate.

In [ ]:
state_fn_o, pdb_fn_o = state_fn, pdb_fn
state_fn, pdb_fn = MR._run_step(state_fn, 1, nsteps=125000, positions_from_pdb=pdb_fn)

For the next few questions, consult the OpenMM [user guide](http://docs.openmm.org/latest/userguide/theory/04_integrators.html) and [documentation](http://docs.openmm.org/latest/api-python/library.html#integrators) about integrators.

#### --> MotorRow uses the LangevinIntegrator from OpenMM. Is the LangevinIntegrator stochastic or deterministic?

#### --> LangevinIntegrator accepts the temperature as an argument. Why?

#### --> Does every integrator accept temperature as an argument? Why or why not?

#### --> With the computer resources that you used, how many nanoseconds of simulation you would be able to perform in a day?

#### --> How does the potential energy of the system compare after minimization and at the beginnig of Step 1? How does it change over the course of Step 1? Why?

In [ ]:
# This visualization shows how Step 1 changes the water in the system
ligand_name = "UNK"

import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'minimized.pdb'), "r").read(),'pdb')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_1.pdb'), "r").read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.setStyle({'resn':ligand_name}, {'stick': {}})
# Show water and lipid near the protein
view.addStyle({'model':0, 'resn':'HOH', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'sphere': {'radius':'0.2', 'color':'red' }})
view.addStyle({'model':1, 'resn':'HOH', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'sphere': {'radius':'0.2', 'color':'blue'}})
view.addStyle({'model':0, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'stick': {'color':'red'}})
view.addStyle({'model':1, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, \
 {'stick': {'color':'blue'}})
view.zoomTo()
view.show()

#### --> How does Step 1 affect the density of water near the protein?

Hint: you may want to visualize the water from model 0 versus model 1.

### Step 2 NVT without restraints

In this step of equilibration, simulation is performed without restraints to allow the protein and membrane to relax.

In [ ]:
state_fn_o, pdb_fn_o = state_fn, pdb_fn
state_fn, pdb_fn = MR._run_step(state_fn, 2, nsteps=125000)

In [ ]:
# This visualization shows how step 2 changes the system
ligand_name = "UNK"

import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_1.pdb'), "r").read(),'pdb')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_2.pdb'), "r").read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.setStyle({'resn':ligand_name}, {'stick': {}})
view.addStyle({'model':0, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'red'}})
view.addStyle({'model':1, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'blue'}})
view.addStyle({'model':0, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'red' }})
view.addStyle({'model':1, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'blue'}})
view.zoomTo()
view.show()

#### --> How does Step 2 affect the system?

### Step 3: NPT with a membrane barostat

This step is performed in the NPT ensemble, with a constant number of particles, pressure, and temperature. Pressure is equilibrated with a barostat, which changes the box volume. The [membrane barostat](http://docs.openmm.org/7.0.0/api-python/generated/simtk.openmm.openmm.MonteCarloMembraneBarostat.html) was designed such that box size in the Z dimension varies independently from other axes.

In [ ]:
#NPT Membrane Barostat
state_fn_o, pdb_fn_o = state_fn, pdb_fn
state_fn, pdb_fn = MR._run_step(state_fn, 3, nsteps=250000)

#### --> Do the box vectors change? In which dimensions is the change the largest?


In [ ]:
# This visualization shows how step 3 changes the system
ligand_name = "UNK"

import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_2.pdb'), "r").read(),'pdb')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_3.pdb'), "r").read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.setStyle({'resn':ligand_name}, {'stick': {}})
view.addStyle({'model':0, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'red'}})
view.addStyle({'model':1, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'blue'}})
view.addStyle({'model':0, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'red' }})
view.addStyle({'model':1, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'blue'}})
view.zoomTo()
view.show()

### Step 4: NPT with a barostat

In [ ]:
state_fn_o, pdb_fn_o = state_fn, pdb_fn
state_fn, pdb_fn = MR._run_step(state_fn, 4, nsteps=250000)

In [ ]:
# This visualization shows how step 4 changes the system
ligand_name = "UNK"

import py3Dmol
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_3.pdb'), "r").read(),'pdb')
view.addModel(open(os.path.join(os.path.dirname(pdb_fn), 'Step_4.pdb'), "r").read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'purple'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.setStyle({'resn':ligand_name}, {'stick': {}})
view.addStyle({'model':0, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'red'}})
view.addStyle({'model':1, 'hetflag':'1', 'within':{'distance':'3', 'atom':'CA'}}, {'stick': {'color':'blue'}})
view.addStyle({'model':0, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'red' }})
view.addStyle({'model':1, 'resn':'HOH'}, {'sphere': {'radius':'0.2', 'color':'blue'}})
view.zoomTo()
view.show()

#### --> Do the box vectors change? In which dimensions is the change the largest?

### Step 5: Longer duration NPT with a barostat

We will run this on a remote supercomputer at a later time.

In [ ]:
# state_fn_o, pdb_fn_o = state_fn, pdb_fn
# state_fn, pdb_fn = MR._run_step(state_fn, 5, nsteps=10000000)

## Discussion

We have successfully performed an MD simulation of a protein ligand complex in membrane. However, we simulated only a short time to keep the execution time of the lab short. To address critical questions in drug design, longer simulations are often required.

Conventional MD simulations are still too computationally costly to be useful for this purpose. Thus, so-called enhanced sampling methods that aim to accelerate the conformational sampling were developed. We will do this in a future exercise.

## Further reading

### Enhanced sampling methods

In theory, unbiased MD simulations should be capable of simulating binding and unbinding events of a drug molecule and its macromolecular target. However, the timescale of binding and unbinding events lies in the millisecond to second range. Enhanced sampling methods aim to accelerate the conformational sampling ([_J Med Chem._ 2016, **59(9)**, 4035-61](https://doi.org/10.1021/acs.jmedchem.5b01684)).

One of these is **Free energy perturbation (FEP)** (also called alchemical free energy calculation), which computes the free energy difference when going from a state A to another state B. It is often employed in lead optimization to evaluate small modification at the ligand, that may boost the binding affinity for the desired target. The ligand from state A is thereby gradually transformed into the ligand of state B by simulating several intermediate ("alchemical") states ([alchemistry](http://www.alchemistry.org/wiki/Main_Page)).

Another technique for free-energy calculations is **Umbrella sampling (US)**. US enforces sampling along a collective variable (CV) by performing staged simulations with an energetic bias. The bias usually takes the form of a harmonic potential, hence the term "umbrella". Its goal is to sample high-energy regions along the CV. However, the use in drug design is limited by the high computational cost.

In contrast, **Steered MD (SMD)** follows a different approach: it applies external forces to the system. Those forces are time-dependent and facilitate the unbinding of the ligand from the target. The SMD calculates the final force exerted on the system. The unbinding force profile can then be used to filter hits from docking calculations and to discriminate active from inactive molecules.

### References

- Review on the impact of MD simulations in drug discovery ([_J Med Chem_ (2016), **59**(9), 4035‐4061](https://doi.org/10.1021/acs.jmedchem.5b01684))
- Review on the physics behind MD simulations and best practices ([_Living J Comp Mol Sci_ (2019), **1**(1), 5957](https://doi.org/10.33011/livecoms.1.1.5957))
- Review on force fields ([_J Chem Inf Model_ (2018), **58**(3), 565-578](https://doi.org/10.1021/acs.jcim.8b00042))
- Review on EGFR in cancer ([_Cancers (Basel)_ (2017), **9**(5), 52](https://dx.doi.org/10.3390%2Fcancers9050052))
- Summarized statistical knowledge from Pierre-Simon Laplace ([Théorie Analytique des Probabilités _Gauthier-Villars_ (1820), **3**)](https://archive.org/details/uvrescompltesde31fragoog/page/n15/mode/2up)
- Inspired by a notebook form Jaime Rodríguez-Guerra ([github](https://github.com/jaimergp/uab-msc-bioinf/blob/master/MD%20Simulation%20and%20Analysis%20in%20a%20Notebook.ipynb))
- Repositories of [OpenMM](https://github.com/openmm/openmm) and [OpenMM Forcefields](https://github.com/openmm/openmmforcefields), [RDKit](https://github.com/rdkit/rdkit), [PyPDB](https://github.com/williamgilpin/pypdb), [MDTraj](https://github.com/mdtraj/mdtraj), [PDBFixer](https://github.com/openmm/pdbfixer)
- Wikipedia articles about [MD simulations](https://en.wikipedia.org/wiki/Molecular_dynamics), [AMBER](https://en.wikipedia.org/wiki/AMBER) and [force fields](https://en.wikipedia.org/wiki/Force_field_(chemistry)) in general